In [1]:
import json
import requests
import pandas as pd
import csv
import os
from kiwipiepy import Kiwi
import re
from collections import Counter
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama

In [4]:
def get_lstrm(page):
    configfile = 'law_config.json'
    law_category = '법률용어'

    with open(configfile, 'r', encoding='utf-8') as f:
        config = json.load(f)

    law_config = config[law_category]

    law_meta = law_config[law_category]

    url = law_meta['url']

    params = {
        'OC' : law_meta['OC'], # 사용자 이메일의 ID(g4c@korea.kr일경우 OC값=g4c) (필수)
        'type' : law_meta['type'], # 출력 형태 HTML/XML/JSON (필수)
        'page' : page, # 검색 결과 페이지 (default=1)
        'display' : 100, # 검색된 결과 개수 (default=20 max=100)
    }

    response = requests.get(url, params=params)
    data = response.json()

    lstrm = data['LsTrmSearch']['lstrm']

    return lstrm

In [99]:
# CSV 헤더 파일 생성

file_name = 'law.csv'

if not os.path.isfile(file_name): # law.csv 파일이 존재하지 않으면
    with open(file_name, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(get_lstrm(1)[0].keys())

In [ ]:
# # CSV 파일에 내용 채우기

# page = 1
# while True:
#     try:
#         lstrm = get_lstrm(page)
#         df = pd.DataFrame.from_dict(lstrm)

#         df.to_csv('law.csv', mode='a', index=False, header=False)
#     except:
#         break

#     page += 1

In [6]:
law = pd.read_excel('작업.xlsx')

law_word = law['법령용어명']

law_word

0        (Instrument Meterological Condition; IMC)
1                (Terminal Operating Company)라 한다)
2                                              (이하
3                  (지리적)위치(Position(geographical))
4                                    0 정지방식 비상정지장치
                           ...                    
72825                                 희석(dilution)
72826                                        희석식소주
72827                                          희석제
72828                                       희석필요지역
72829                        희석필요지역(dilution area)
Name: 법령용어명, Length: 72830, dtype: object

In [7]:
kiwi = Kiwi()

corpus_set = set()

for word in law_word:

    noun_string = ''

    token_list = kiwi.tokenize(str(word))

    for token in token_list:
        if token.tag in ['NNG', 'NNP', 'XSN', 'XPN', 'NR']:
            noun_string += token.form
        else:
            noun_string += ' '

    noun_list = noun_string.split()

    for noun in noun_list:
        corpus_set.add(noun)

In [8]:
corpus = sorted(corpus_set)
# corpus

In [9]:
corpusDF = pd.DataFrame(corpus, columns=['법령용어'])
corpusDF.to_csv('corpus.csv', index=False, header=False)

In [3]:
kiwi = Kiwi()

kiwi.tokenize('초일불산입')

[Token(form='초', tag='XPN', start=0, len=1),
 Token(form='일', tag='NR', start=1, len=1),
 Token(form='불', tag='NNG', start=2, len=1),
 Token(form='산입', tag='NNG', start=3, len=2)]

In [31]:
llm = Ollama(model = 'exaone3.5:7.8b')

prompt = """
지금 우리는 법령용어 코퍼스 데이터를 수집하고 있어.
'{word}' 이 단어가 코퍼스 데이터로 사용하기에 적합하면 O를, 아니면 X를 답변해줘.
"""

for index, word in enumerate(corpus[123:]):
    answer = llm.invoke(prompt)

    if 'O' in answer:
        with open('final_corpus.csv', 'a', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([word])
    
        print(f"index {index} : {word} (완료)")

index 0 : 가등기명의 (완료)
index 1 : 가등기설정계약 (완료)
index 2 : 가등기의무 (완료)
index 3 : 가등록 (완료)
index 4 : 가로 (완료)
index 5 : 가로구역 (완료)
index 6 : 가로등 (완료)
index 7 : 가로수 (완료)
index 8 : 가로중심점 (완료)
index 9 : 가맹계약 (완료)
index 10 : 가맹계약서 (완료)
index 11 : 가맹금 (완료)
index 12 : 가맹본부 (완료)
index 13 : 가맹사업 (완료)
index 14 : 가맹사업시준수사항 (완료)
index 15 : 가맹사업자 (완료)
index 16 : 가맹시장 (완료)
index 17 : 가맹유통 (완료)
index 18 : 가맹점 (완료)
index 19 : 가맹점모집인 (완료)
index 20 : 가맹점사업자 (완료)
index 21 : 가맹점수수료율 (완료)
index 22 : 가맹점운영권 (완료)
index 23 : 가맹중개인 (완료)
index 24 : 가맹지역본부 (완료)
index 25 : 가맹희망자 (완료)
index 26 : 가명 (완료)
index 27 : 가명정보 (완료)
index 28 : 가명정보결합전문기관 (완료)
index 29 : 가명정보보유기관 (완료)
index 30 : 가명정보처리 (완료)
index 31 : 가명조서 (완료)
index 32 : 가명진술서 (완료)
index 33 : 가명처리 (완료)
index 34 : 가목 (완료)
index 35 : 가문 (완료)
index 36 : 가뭄 (완료)
index 37 : 가변성 (완료)
index 38 : 가변속제어기 (완료)
index 39 : 가변적수명관리프로그램 (완료)
index 40 : 가변차로 (완료)
index 42 : 가부동수 (완료)
index 43 : 가분채권 (완료)
index 44 : 가분채무 (완료)
index 45 : 가불금 (완료)
index 46 : 가사 (완료)
index 47 : 가사근로

KeyboardInterrupt: 

O

'어패류'는 법령이나 관련 문서에서 해양생물, 특히 물고기와 조개류 등을 지칭하는 용어로 사용될 수 있습니다. 따라서 법령용어 코퍼스 데이터로서 관련성 있고 적합한 단어입니다.
